In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import sklearn
import os
import sys
import time
import tensorflow as tf
from tensorflow import keras

print (tf.__version__)
print (sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print (module.__name__, module.__version__)

2.0.0-beta1
sys.version_info(major=3, minor=6, micro=8, releaselevel='final', serial=0)
matplotlib 3.1.1
numpy 1.16.4
pandas 0.24.2
sklearn 0.21.2
tensorflow 2.0.0-beta1
tensorflow.python.keras.api._v2.keras 2.2.4-tf


In [3]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
print (housing.DESCR)
print (housing.data.shape)
print (housing.target.shape)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block
        - HouseAge      median house age in block
        - AveRooms      average number of rooms
        - AveBedrms     average number of bedrooms
        - Population    block population
        - AveOccup      average house occupancy
        - Latitude      house block latitude
        - Longitude     house block longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
http://lib.stat.cmu.edu/datasets/

The target variable is the median house value for California districts.

This dataset was derived from the 1990 U.S. census, using one row per census
block group. A block group is the smallest geographical unit for which the U.S.
Census Bur

In [4]:
import pprint
pprint.pprint (housing.data[0:5])
pprint.pprint(housing.target[0:5])

array([[ 8.32520000e+00,  4.10000000e+01,  6.98412698e+00,
         1.02380952e+00,  3.22000000e+02,  2.55555556e+00,
         3.78800000e+01, -1.22230000e+02],
       [ 8.30140000e+00,  2.10000000e+01,  6.23813708e+00,
         9.71880492e-01,  2.40100000e+03,  2.10984183e+00,
         3.78600000e+01, -1.22220000e+02],
       [ 7.25740000e+00,  5.20000000e+01,  8.28813559e+00,
         1.07344633e+00,  4.96000000e+02,  2.80225989e+00,
         3.78500000e+01, -1.22240000e+02],
       [ 5.64310000e+00,  5.20000000e+01,  5.81735160e+00,
         1.07305936e+00,  5.58000000e+02,  2.54794521e+00,
         3.78500000e+01, -1.22250000e+02],
       [ 3.84620000e+00,  5.20000000e+01,  6.28185328e+00,
         1.08108108e+00,  5.65000000e+02,  2.18146718e+00,
         3.78500000e+01, -1.22250000e+02]])
array([4.526, 3.585, 3.521, 3.413, 3.422])


In [5]:
from sklearn.model_selection import train_test_split

x_train_all, x_test, y_train_all, y_test = train_test_split(
    housing.data, housing.target, random_state = 7, test_size=0.25)
x_train, x_valid, y_train, y_valid = train_test_split(
    x_train_all, y_train_all, random_state=11, test_size = 0.25)

print (x_train.shape, y_train.shape)
print (x_valid.shape, y_valid.shape)
print (x_test.shape, y_test.shape)

(11610, 8) (11610,)
(3870, 8) (3870,)
(5160, 8) (5160,)


In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)  #这里使用fit_transform函数，因为在训练集上获得均值和方差，在验证集和测试集上面使用
x_valid_scaled = scaler.transform(x_valid)
x_test_scaled = scaler.transform(x_test)

In [15]:
output_dir = "generate_csv"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

'''
n_parts: tf.dataset可以将多个文件merge起来形成一个数据集，为了测试这一功能，所以这里我们存成多个文件。
header: csv文件可能有header信息
'''
def save_to_csv(output_dir, data, name_prefix,
                header=None, n_parts=10):
    path_format = os.path.join(output_dir,"{}_{:02d}.csv") #定义了文件路径格式
    filenames = []
    
    #将与数据集等长的序列分为n_parts个列表，根据每个序列的index作为文件名，序列里的value作为索引读取data数据
    for file_idx, row_indices in enumerate(
        np.array_split(np.arange(len(data)),n_parts)):
        part_csv = path_format.format(name_prefix, file_idx)
        filenames.append(part_csv)
        with open(part_csv, "wt", encoding = "utf-8") as f:
            if header is not None:
                f.write(header +"\n")
            for row_index in row_indices:
                f.write(",".join(
                    [repr(col) for col in data[row_index]]))
                f.write('\n')        
    return filenames

## 这段代码将数据 x、y按行merge起来，并获取header信息
train_data = np.c_[x_train_scaled, y_train] #np.c_ 按行merge
valid_data = np.c_[x_valid_scaled, y_valid]
test_data = np.c_[x_test_scaled, y_test]
header_cols = housing.feature_names +["MidianHouseValue"]
header_str = ",".join(header_cols)#将元素用','隔开

train_filenames = save_to_csv(output_dir, train_data, "train", header_str, n_parts=20)
valid_filenames = save_to_csv(output_dir, valid_data, "valid", header_str,n_parts=10 )
test_filenames = save_to_csv(output_dir, test_data, "test",header_str, n_parts=10)

In [16]:
print (housing.feature_names)
print (header_str)

['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MidianHouseValue


In [17]:
##读取csv文件进行训练

In [18]:
import pprint 
print ("train filenames")
pprint.pprint(train_filenames)


train filenames
['generate_csv/train_00.csv',
 'generate_csv/train_01.csv',
 'generate_csv/train_02.csv',
 'generate_csv/train_03.csv',
 'generate_csv/train_04.csv',
 'generate_csv/train_05.csv',
 'generate_csv/train_06.csv',
 'generate_csv/train_07.csv',
 'generate_csv/train_08.csv',
 'generate_csv/train_09.csv',
 'generate_csv/train_10.csv',
 'generate_csv/train_11.csv',
 'generate_csv/train_12.csv',
 'generate_csv/train_13.csv',
 'generate_csv/train_14.csv',
 'generate_csv/train_15.csv',
 'generate_csv/train_16.csv',
 'generate_csv/train_17.csv',
 'generate_csv/train_18.csv',
 'generate_csv/train_19.csv']


In [19]:
# filename ->dataset
# read file ->dataset --> datasets -> merge -->
# 解析csv

#list_files:处理文件名，生成tensor
filename_dataset = tf.data.Dataset.list_files(train_filenames)
for filename in filename_dataset:
    print (filename)

tf.Tensor(b'generate_csv/train_18.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_01.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_12.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_03.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_08.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_09.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_07.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_02.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_00.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_04.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_10.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_17.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_15.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_14.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_16.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_13.csv', 

In [22]:
#按行读取文件文件的API TextLineDataset
n_readers = 5 #并行度
dataset = filename_dataset.interleave(
    lambda filename: tf.data.TextLineDataset(filename).skip(1), #skip:跳过header行，这里跳过一行
    cycle_length = n_readers
)

#take: 选择前n个
for line in dataset.take(15):
    print(line.numpy())

b'-1.1199749330438333,-1.329843308393715,0.1419004518620726,0.4658136987980791,-0.10301777467500105,-0.10744184416176107,-0.7950524078397521,1.5304716763409,0.66'
b'0.04971034572063198,-0.8492418886278699,-0.06214699417830008,0.17878747064657746,-0.8025354230744277,0.0005066066922077538,0.6466457006743215,-1.1060793768010604,2.286'
b'-1.0775077698160966,-0.44874070548966555,-0.5680568205591913,-0.14269262164909954,-0.09666677138213985,0.12326468238687088,-0.3144863716683942,-0.4818958888413162,0.978'
b'-0.32652634129448693,0.43236189741438374,-0.09345459539684739,-0.08402991822890092,0.8460035745154013,-0.0266316482653991,-0.5617679242614233,0.1422875991184281,2.431'
b'0.6303435674178064,1.874166156711919,-0.06713214279531016,-0.12543366804152128,-0.19737553788322462,-0.022722631725889016,-0.692407235065288,0.7265233438487496,2.419'
b'-0.9490938885377456,0.6726626072973063,0.28370554761513944,0.10655529643465292,-0.6546477749692311,-0.0623949278698749,0.21273656121863005,0.002470497815

In [24]:
'''
demo展示
'''
# tf.io.decode_csv(str, record_defaults):解析csv文件 record_defaults定义各个field的类型是什么样的
sample_str = "1,2,3,4,5"
record_defaults = [tf.constant(0,dtype = tf.int32)]*5
#print (record_defaults)

parsed_fields = tf.io.decode_csv(sample_str, record_defaults)
print (parsed_fields)

[<tf.Tensor: id=103, shape=(), dtype=int32, numpy=1>, <tf.Tensor: id=104, shape=(), dtype=int32, numpy=2>, <tf.Tensor: id=105, shape=(), dtype=int32, numpy=3>, <tf.Tensor: id=106, shape=(), dtype=int32, numpy=4>, <tf.Tensor: id=107, shape=(), dtype=int32, numpy=5>]


In [25]:
# 空字符串
try:
    parsed_fields = tf.io.decode_csv(',,,,', record_defaults)
except tf.errors.InvalidArgumentError as ex:
    print (ex)

In [28]:
def parse_csv_line(line, n_fields=9):
    defs = [tf.constant(np.nan)]*n_fields
    parsed_fields = tf.io.decode_csv(line, record_defaults=defs)
    x = tf.stack(parsed_fields[0:-1]) #tf.stack:转化为向量
    y = tf.stack(parsed_fields[-1:])
    return x,y
parse_csv_line(b'-1.1199749330438333,-1.329843308393715,0.1419004518620726,0.4658136987980791,-0.10301777467500105,-0.10744184416176107,-0.7950524078397521,1.5304716763409,0.66',9)

(<tf.Tensor: id=234, shape=(8,), dtype=float32, numpy=
 array([-1.119975  , -1.3298433 ,  0.14190045,  0.4658137 , -0.10301778,
        -0.10744184, -0.7950524 ,  1.5304717 ], dtype=float32)>,
 <tf.Tensor: id=235, shape=(1,), dtype=float32, numpy=array([0.66], dtype=float32)>)

In [27]:
def csv_reader_dataset(filenames, n_readers=5,
                      batch_size=32, n_parse_treads=5,
                      shuffle_buffer_size=10000):
    dataset = tf.data.Dataset.list_files(filenames)
    dataset = dataset.repeat() #重复无限次
    dataset = dataset.interleave(
        lambda filename: tf.data.TextLineDataset(filename).skip(1),
        cycle_length = n_readers
    )
    dataset.shuffle(shuffle_buffer_size)
    #解析的工作
    dataset = dataset.map(parse_csv_line,
                         num_parallel_calls=n_parse_treads)
    dataset = dataset.batch(batch_size)
    return dataset
train_set = csv_reader_dataset(train_filenames, batch_size=3)
for x_batch, y_batch in train_set.take(2):
    print("x:")
    pprint.pprint(x_batch)
    print("y:")
    pprint.pprint(y_batch)

x:
<tf.Tensor: id=215, shape=(3, 8), dtype=float32, numpy=
array([[-1.1199750e+00, -1.3298433e+00,  1.4190045e-01,  4.6581370e-01,
        -1.0301778e-01, -1.0744184e-01, -7.9505241e-01,  1.5304717e+00],
       [-8.2195884e-01,  1.8741661e+00,  1.8212350e-01, -3.1700194e-02,
        -6.0111791e-01, -1.4337493e-01,  1.0852206e+00, -8.6139947e-01],
       [ 4.9710345e-02, -8.4924191e-01, -6.2146995e-02,  1.7878747e-01,
        -8.0253541e-01,  5.0660671e-04,  6.4664572e-01, -1.1060793e+00]],
      dtype=float32)>
y:
<tf.Tensor: id=216, shape=(3, 1), dtype=float32, numpy=
array([[0.66 ],
       [1.054],
       [2.286]], dtype=float32)>
x:
<tf.Tensor: id=219, shape=(3, 8), dtype=float32, numpy=
array([[ 0.40127665, -0.92934215, -0.0533305 , -0.18659453,  0.65456617,
         0.02643447,  0.9312528 , -1.4406418 ],
       [ 0.04326301, -1.0895426 , -0.38878718, -0.10789865, -0.68186635,
        -0.0723871 , -0.8883662 ,  0.8213992 ],
       [-0.9490939 ,  0.6726626 ,  0.28370556,  0.1065553 

In [13]:
for file_idx, row_indices in enumerate(np.array_split(np.arange(20),4)):
    print (file_idx, row_indices)

0 [0 1 2 3 4]
1 [5 6 7 8 9]
2 [10 11 12 13 14]
3 [15 16 17 18 19]


In [ ]:
output_dir = 